In [4]:
import numpy as np
from scipy.optimize import bisect

# Model

Preferences are given by : 

$$ U(c,h) = \frac{c^{1-\sigma_c}}{1-\sigma_c} + \phi \frac{h^{1-\sigma_h}}{1-\sigma_h} $$

The production function is 

$$h = \alpha_1 + \alpha_0 m$$

The budget constraint is such that 

$$y = c + p m$$

There are two regions, us and eu. Upon substituting $h$, we have the following problem: 

$$ \max_{m} \frac{(y-pm)^{1-\sigma_c}}{1-\sigma_c} + \phi \frac{(\alpha_1 + \alpha_0m)^{1-\sigma_h}}{1-\sigma_h} $$

The FOC of this problem is 

$$ p(y - pm)^{-\sigma_c} = \alpha_0 \phi (\alpha_1 + \alpha_0 m)^{-\sigma_h} $$

We first program up the solution, using bisection (no derivative involved)

In [13]:
def foc(m,sigma_c,sigma_h,phi,alphas,y,p):
	return p*(y-p*m)**(-sigma_c) - alphas[0]*phi*(alphas[1]+alphas[0]*m)**(-sigma_h)
def optm(sigma_c,sigma_h,phi,alphas,y,p):
	return bisect(foc,0.01*y,y*0.95,args=(sigma_c,sigma_h,phi,alphas,y,p))

These are the parameters that we end up getting, but we check here that the function works. 

In [14]:
alphas_us = [6.66,0]
sigma_c = 2
sigma_h = 3
phi = 0.207

We start with income and price normalized in U.S. 

In [ ]:
p_us = 1
y_us = 1

We can check that the function returns s in the U.S. 

In [15]:
optm(sigma_c,sigma_h,phi,alphas_us,y_us,p_us)

0.14995735503423932

In [19]:
s_us = 0.15
h_us = 1.0

In [16]:
alphas_eu = [6.66,0.22]
p_eu = 0.61
y_eu = 0.78

In [18]:
p_eu*optm(sigma_c,sigma_h,phi,alphas_eu,y_eu,p_eu)/y_eu

0.09622527852617895

In [20]:
s_eu = 0.096
h_eu = 1.047

In [22]:
def uopt(sigma_c,sigma_h,phi,alphas,y,p):
	m = optm(sigma_c,sigma_h,phi,alphas,y,p)
	return ((y-p*m)**(1-sigma_c))/(1-sigma_c) + phi*((alphas[1]+alphas[0]*m)**(1-sigma_h))/(1-sigma_h) 
def prod(alphas,m):
	return alphas[1] + alphas[0]*m
def hopt(sigma_c,sigma_h,phi,alphas,y,p):
	m = optm(sigma_c,sigma_h,phi,alphas,y,p)
	return prod(alphas,m)

In [23]:
hopt(sigma_c,sigma_h,phi,alphas_eu,y_eu,p_eu)

1.0394607817832697

# Calibration

We start the exercise with finding U.S. parameters that fit the moments. We are solving a set of equations for two unknowns, $\alpha_0$ and $\phi$. We use the methods of moments to minimize the squared distance of moments. We use a derivative free algorithm 

In [24]:
def criterion_us(theta,sigma_c,sigma_h,alphas,y,p,moms):
	alphas[0] = theta[0]
	phi = theta[1]
	m = optm(sigma_c,sigma_h,phi,alphas,y,p)
	moms_s = np.zeros(2)
	moms_s[0] = p*m/y
	moms_s[1] = prod(alphas,m)
	return (moms[0] - moms_s[0])**2 + (moms[1] - moms_s[1])**2

These are the numbers presented in Section 2

In [29]:
from scipy.optimize import minimize

opt_us = minimize(criterion_us,[5.0,0.3],args=(sigma_c,sigma_h,[0.0,0.0],y_us,p_us,[s_us,h_us]))
alphas_us[0] = opt_us.x[0]
phi = opt_us.x[1]
alphas_us[0],phi


(6.666657902257493, 0.20761274012255118)

In Europe, we seek to estimate $p_{EU}$ and $\alpha_{1,EU}$ using $(s_{EU},h_{EU})$ as moments

In [30]:
def criterion_eu(theta,sigma_c,sigma_h,phi,alphas,y,moms):
	alphas[1] = theta[0]
	p = theta[1]
	m = optm(sigma_c,sigma_h,phi,alphas,y,p)
	moms_s = np.zeros(2)
	moms_s[0] = p*m/y
	moms_s[1] = prod(alphas,m)
	return (moms[0] - moms_s[0])**2 + (moms[1] - moms_s[1])**2

These are the numbers reported in Section 2

In [33]:
alphas_eu[0] = alphas_us[0]
opt_eu = minimize(criterion_eu,[0.4,0.8],args=(sigma_c,sigma_h,phi,alphas_eu,y_eu,[s_eu,h_eu]))
alphas_eu[1] = opt_eu.x[0]
p_eu = opt_eu.x[1]
alphas_eu[1],p_eu

(0.21439909063306478, 0.599575021285484)